In [21]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import norm
from scipy import stats

In [22]:
training_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
sample_submission=open('sample_submission.csv')

In [24]:
passenger_ids=[]
for pos in test_data.index:
    passenger_ids.append(test_data['PassengerId'][pos])
    
print(len(passenger_ids))

4277


In [25]:
#fill null values
age_median=training_data['Age'].median()
training_data['Age'].fillna(age_median,inplace=True)
test_data['Age'].fillna(age_median,inplace=True)

fill_with_most_frequent=['CryoSleep','ShoppingMall','VIP','HomePlanet','VRDeck','Spa','FoodCourt','Destination','RoomService']
for category in fill_with_most_frequent:
    training_data[category].fillna(training_data[category].mode()[0],inplace=True)
    test_data[category].fillna(training_data[category].mode()[0],inplace=True)
    
training_data['Cabin'].fillna('Un/-1/Un',inplace=True) #filler just meaning unknown cabin, format is deck/num/side
test_data['Cabin'].fillna('Un/-1/Un',inplace=True)

training_data.drop('Name', axis=1, inplace=True)
test_data.drop('Name',axis=1,inplace=True)
training_data.drop('PassengerId',axis=1,inplace=True)
test_data.drop('PassengerId',axis=1,inplace=True)

In [26]:
deck_values,deck_values_t=[],[]
cabin_numbers,cabin_numbers_t=[],[]
side_values,side_values_t=[],[]
#separate out deck and side from cabin numbers
for pos in training_data.index:
    cabin_number=training_data['Cabin'][pos]
    cabin_number=cabin_number.split('/')
    deck_values.append(cabin_number[0])
    cabin_numbers.append(int(cabin_number[1]))
    side_values.append(cabin_number[2])
    
for pos in test_data.index:
    cabin_number=test_data['Cabin'][pos]
    cabin_number=cabin_number.split('/')
    deck_values_t.append(cabin_number[0])
    cabin_numbers_t.append(int(cabin_number[1]))
    side_values_t.append(cabin_number[2])
    
training_data.drop('Cabin',axis=1,inplace=True)
test_data.drop('Cabin',axis=1,inplace=True)


training_data['Deck']=deck_values
training_data['CabinNumber']=cabin_numbers
training_data['Side']=side_values
test_data['Deck']=deck_values_t
test_data['CabinNumber']=cabin_numbers_t
test_data['Side']=side_values_t

In [27]:
#replacing values with numerical representations
training_data['Transported']=training_data.Transported.replace('False',0).replace('True',1).astype(np.int)
test_data['Transported']=training_data.Transported.replace('False',0).replace('True',1).astype(np.int)
training_data['CryoSleep']=training_data.CryoSleep.replace('False',0).replace('True',1).astype(np.int)
test_data['CryoSleep']=training_data.CryoSleep.replace('False',0).replace('True',1).astype(np.int)
training_data['VIP']=training_data.VIP.replace('False',0).replace('True',1).astype(np.int)
test_data['VIP']=training_data.VIP.replace('False',0).replace('True',1).astype(np.int)

In [28]:
#one hot encoding
one_hot_encode_cols=training_data.dtypes[training_data.dtypes==np.object]
one_hot_encode_cols=one_hot_encode_cols.index.tolist()
training_data=pd.get_dummies(training_data,columns=one_hot_encode_cols,drop_first=True)

one_hot_encode_cols=test_data.dtypes[test_data.dtypes==np.object]
one_hot_encode_cols=one_hot_encode_cols.index.tolist()
test_data=pd.get_dummies(test_data,columns=one_hot_encode_cols,drop_first=True)

In [29]:
feature_cols=[x for x in training_data.columns if x != 'Transported']
#train_idx,test_idx=training_data[feature_cols],training_data['Transported']

#create datasets
x_train = training_data[feature_cols]
y_train = training_data['Transported']
x_test = test_data[feature_cols]

In [30]:
import warnings
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)

from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(oob_score=True,random_state=42,warm_start=True,n_jobs=-1,n_estimators=200)
RF.fit(x_train,y_train)
y_pred=RF.predict(x_test)

In [38]:
output_file=open('kaggle_final_submission.csv','w')
output_file.write('PassengerId,Transported\n')

for pos in range(len(passenger_ids)):
    outcome='False'
    if y_pred[pos]==1:
        outcome='True'
    output_file.write(passenger_ids[pos]+','+outcome+'\n')
    #print(pos)

In [ ]:
#final score according to kaggle is 0.76993 for accuracy